In [1]:
import torch
from torchviz import make_dot
from torchvision import transforms

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

import time
from PIL import Image
import requests
from io import BytesIO
from tqdm import tqdm


vits16 = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
vits16.eval()



Using cache found in /home/aagrawal/.cache/torch/hub/facebookresearch_dino_main


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (head): Identity()
)

# Identifying Quantizable Components

For a model like a Vision Transformer, the main quantizable components typically include:

Linear layers (torch.nn.Linear): These are already being quantized in your current setup.
Convolutional layers (torch.nn.Conv2d): This layer can be quantized as it operates in a manner similar to linear layers but with spatial data.

# Additional Quantization Considerations

Quantizing other types of layers, especially normalization and dropout, is typically not done or not beneficial:

Normalization layers (e.g., LayerNorm): Quantizing these layers can disrupt the model's internal statistics and generally does not lead to computational savings.
Dropout layers: These are non-operative in inference mode (i.e., they do nothing during inference) and therefore do not need quantization.
Activation functions like GELU: Quantizing activations is more complex and typically involves using quantization-aware training rather than post-training quantization to maintain accuracy.

In [2]:
'''
Dynamic quantisation

Dynamic quantization is a useful technique for reducing the memory footprint and potentially speeding up 
inference, especially on certain hardware. However, applying dynamic quantization directly to a Vision 
Transformer model like vits16 from the DINO project can be challenging due to the complex nature of 
transformer architectures.
'''

# Apply dynamic quantization only to the linear layers
quantized_model_1 = torch.quantization.quantize_dynamic(
    vits16,
    {torch.nn.Linear},
    dtype=torch.qint8
)

quantized_model_2 = torch.quantization.quantize_dynamic(
    vits16,
    {torch.nn.Linear, torch.nn.Conv2d}, 
    dtype=torch.qint8 
)

In [3]:
'''
Inspect the model

'''

print(f" 🍩 Original model: {vits16}\n\n")
print(f" 🚀 Quantized model 1: {quantized_model_1}")
print(f" 🌟 Quantized model 2: {quantized_model_1}")

# Create a dummy input that matches the input size of the model
dummy_input = torch.randn(1, 3, 224, 224)

# Forward pass through the model to get the output
model_output = vits16(dummy_input)

# Visualize the model
dot = make_dot(model_output, params=dict(vits16.named_parameters()))
dot.render('model_visualization', format='png')

 🍩 Original model: VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (hea

'model_visualization.png'

In [4]:
# Define the image transformations
transform = transforms.Compose([
    transforms.Resize(256),       # Resize the larger side of the image to 256
    transforms.CenterCrop(224),   # Crop the center 224x224 pixels
    transforms.ToTensor(),        # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet's mean and std
])

# Load an image from a URL or a local path
def load_image(image_path):
    # If the path is a URL, load it from the web
    if image_path.startswith('http'):
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content))
    else:
        # Load image from local storage
        image = Image.open(image_path)
    image = image.convert('RGB')  # Convert the image to RGB
    return image

# Path or URL to the image
image_path = 'https://cdn.theatlantic.com/thumbor/vDZCdxF7pRXmZIc5vpB4pFrWHKs=/559x0:2259x1700/1080x1080/media/img/mt/2017/06/shutterstock_319985324/original.jpg'  # Change this to your image path or URL
image = load_image(image_path)

# Preprocess the image and add batch dimension (model expects batches)
image = transform(image).unsqueeze(0)  # Add batch dimension

# Perform inference
models = [vits16, quantized_model_1, quantized_model_2]

for model in models:
    tic = time.time()
    with torch.no_grad():  # Deactivate autograd for inference
        output = model(image)
    toc = time.time()
    print(f'Inference time: {toc - tic:.2f}s')
    print(output.shape)  # Output shape: (batch_size, num_patches, feature_dim)


Inference time: 5.21s
torch.Size([1, 384])
Inference time: 4.10s
torch.Size([1, 384])
Inference time: 4.12s
torch.Size([1, 384])


### Note

Dynamic quantization is simple and does not require retraining, but it's generally less effective at reducing model size and improving speed compared to static quantization or quantization-aware training. For layers like activations or any custom operations, consider exploring quantization-aware training for comprehensive quantization while maintaining accuracy.

In [5]:
model = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
model.eval()
model.cuda()

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match the DINO model's expected input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the Tiny ImageNet dataset
train_dataset = ImageFolder('tiny-imagenet-200/tiny-imagenet-200/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Perform inference on the entire dataset\
total = 0
correct = 0
total_inference_time = 0

# models = [vits16]

for img, labels in tqdm(train_loader):
    tic = time.time()
    with torch.no_grad():
        img = img.cuda()
        labels = labels.cuda()
        outputs = model(img)
        toc = time.time()
        inference_time = toc - tic
        total_inference_time += inference_time

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        print(f"predicted: {predicted[0]}, actual: {labels[0]}")

average_inference_time = total_inference_time / len(train_loader)
accuracy = correct / total



# Calculate accuracy and average inference time on the validation set
print(f'Accuracy on the train set: {accuracy:.2f}')
print(f'Average inference time per batch: {average_inference_time:.4f} seconds')



Using cache found in /home/aagrawal/.cache/torch/hub/facebookresearch_dino_main
  0%|          | 1/3125 [00:06<5:25:53,  6.26s/it]

predicted: 319, actual: 187


  0%|          | 2/3125 [00:11<4:56:15,  5.69s/it]

predicted: 168, actual: 41


  0%|          | 3/3125 [00:16<4:46:18,  5.50s/it]

...

predicted: 22, actual: 132


 46%|████▋     | 1449/3125 [2:06:31<2:26:58,  5.26s/it]

predicted: 94, actual: 178


 46%|████▋     | 1450/3125 [2:06:37<2:27:03,  5.27s/it]

predicted: 280, actual: 182
